Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 2 </h2>
    <h1> Ham o spam  </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Segundo Semestre 2022<br>    
        Fecha de entrega: Viernes 23 de septiembre 22.00 horas
    </p>
    <br>
</center>

<br>

---

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas. 

**IMPORTANTE**: 
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.


Utilizaremos una base de datos ubicada en Kaggle https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset que puedes encontrar igualmente en canvas como csv para descargar.

## Introducción

Nadie es inmune a recibir mensajes de Movistar o Entel ofreciéndonos planes. Abrir un mensaje para encontrarse con una hermosa sorpresa: es spam. 

Este es un problema a nivel mundial, tanto así que se han armado bases de datos con diferentes mensajes de texto recibidos por persona y si son considerados como spam o no (si no son spam se refiere a los mensajes como ham).

Utilizando la vectorización de frases y clusterizando estas, deberás predecir si esta es o no spam. Además, deberás obtener los índices de calidad de los clusters.

## 0. Setup

In [1]:
from IPython.display import clear_output
clear_output()
print('Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda')

Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda


## 1. Importar librerías y descargar dataset
En esta tarea trabajaremos con la librería `spacy` y el pipeline `en_core_web_lg` el cual pesa más de 500 MB y contiene un vocabulario en inglés de más de medio millón de palabras. Cada una de estas palabras es representable a partir de un vector de 300 dimensiones que nos ayudarán en la tarea. Revisa la [documentación](https://spacy.io/api) documentación de la librería para saber más.

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import spacy

nlp = spacy.load("en_core_web_lg")

### Leer dataset

In [3]:
url = 'spam.csv'
df = pd.read_csv(url, index_col=0, encoding="ISO-8859-1")
df.reset_index(inplace=True)
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


## 2. Procesamiento de los datos (1 punto)

### 2.1 Eliminación de datos
Solamente analizaremos las columnas de si es o no spam y cuál es el mensaje. Elimina las columnas restantes y preprocesa las filas eliminando los valores nulos.

Primero eliminamos las columnas Unnamed: 2, Unnamed: 3 y Unnamed 4. Luego eliminaremos los valores nulos

In [4]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df = df.dropna()
df.head(50)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


### 2.2 Preprocesamiento de oraciones

Acá te damos el código para preprocesar un texto.

In [5]:
import string

all_stopwords = nlp.Defaults.stop_words

def remove_punctuation(text):
  text = [token for token in text if not token.is_punct]
  return text

def remove_stopwords(words):
  words = [word for word in words if not word in all_stopwords]
  return words

def lemmatize(words):
  words = [word.lemma_ for word in words]
  return words

def remove_non_alpha(words):
  words = [word for word in words if word.isalpha()]
  return words

def lower(words):
  words = [word.lower() for word in words]
  return words

def min_len(words, length=3):
  words = [word for word in words if len(word)>=length]
  return words

def preprocess(text):

  doc = nlp(text)
  tokens = remove_punctuation(doc)
  tokens = remove_stopwords(tokens)
  tokens = lemmatize(tokens)
  tokens = remove_non_alpha(tokens)
  tokens = lower(tokens)
  tokens = min_len(tokens, length=3)

  return ' '.join(tokens).strip()

# Este es un ejemplo para que veas si tu preprocesamiento funcionó.
new_text = preprocess("This is the 2nd time we have tried 2 contact...")
new_text

'this the time have try contact'

Preprocesa todos los mensajes utilizando el método preprocess y guárdalos en un dataframe.

In [ ]:
df_p = df['v2'].apply(func=preprocess)

df_p

### 2.3 Vectorizar oraciones
En esta tarea, el vector de una oración será el promedio de los vectores de cada una de las palabras que fueron preprocesadas de la oración. La función presentada a continuación vectoriza una oración a partir de los vectores de las palabras.



In [ ]:
def sentence_vector(text):
  text = nlp(text)
  vectores = []
  for t in text:
    t_vector = t.vector
    vectores.append(t_vector)
  return np.array(vectores).sum(axis=0)/len(vectores)

In [ ]:
print(list_v)

### 2.4. Obtener matriz de distancias


Obtén una forma de calcular una matriz que por cada par distintos de oraciones contenga la distancia euclidiana y coseno entre los vectores que representan a cada una. 

Hint: el método pairwise_distances de sklearn realiza esta operación eficientemente y no genera problemas de RAM.



In [ ]:
from sklearn.metrics import pairwise_distances
matriz = pairwise_distances(sentence_vector(df_p['v2']), sentence_vector(df_p['v2']), metric='euclidean')

# Tu código aquí

In [ ]:
list_v = []
for i in df_p['v2']:
    list_v.append(sentence_vector(i))
    print(sentence_vector(i))


## 3. Clase AgglomerativeClustering (3 puntos)

Esta clase debe implemetar el algoritmo de clustering jerárquico aglomerativo. Para esto **debes** programar los siguientes métodos:


### **\_\_init\_\_**
Inicializa el algoritmo a partir de: la matriz `X` de los mensajes y la matriz de distancias. Considera además todas las variables que necesites a lo largo de la ejecución de tu algoritmo, se recomienda como mínimo:
*   Un contador que indique el nivel actual de aglomeración.
*   Un diccionario o lista que almacene los clusters en cada nivel de aglomeración. Inicialmente, en el nivel 0, existe un *cluster* por cada mensaje de `X`.
*   La matriz de distancia entre los *clusters* donde el elemento `matriz[id1][id2]` corresponde a la distancia entre los clusters con identificadores `id1` e `id2` respectivamente. 
*   Una copia de la matriz original X.


### **clusterize**
Ejecuta el método next_level hasta que solo existan dos *clusters*.

##### **next_level**
Equivale a realizar un nivel de aglomeración del algoritmo. A modo general deben:
1.   Obtener el par de *clusters* con menor distancia a partir de la matriz de distancias obtenida en 3.
2.   Unir ambos *clusters*.
3.   Guardar el nuevo conjunto de *clusters* correspondientes al nivel actual de aglomeración.
4.   Actualizar la matriz de distancias según el nuevo conjunto de *clusters*.

##### **update_matrix**
Actualiza la matriz de distancias dado un nuevo *cluster*. La distancia entre *clusters* debe poder calcularse según los siguientes enlaces (`linkage`) vistos en clases:
1.   **centroid**: distancia entre medias.
2.   **single**: simple.

<br>

---

**NOTA**: puedes entregarle los argumentos que quieras a estos métodos y tambien crear otros métodos que consideres pertinentes.

En base a lo visto en clases deberás implementar el algoritmo de clustering aglomerativo para agrupar los datos previamente preprocesados.

In [ ]:
class AgglomerativeClustering:

  def __init__(self, X, linkage="centroid", distance="Euclidean"):
    # Vectores de cada pais.
    self.X = X.copy()
    # Linkage.
    self.linkage = linkage
    # Métrica de distancia.
    self.distance = distance

    # Matriz de distancias (obtén la matriz de distancias inicial)
    # self.matrix = # 

    # Copia de la matriz de distancias original.
    self.original_matrix = self.matrix.copy()


  def clusterize(self):
    pass

  def next_level(self):
    # Obtén el par de clusters con menor distancia de la matriz de distancias.
    
    # Crea un nuevo cluster a partir de los dos anteriores
    
    # El nuevo nivel tiene los clusters anteriores y la union de los dos clusters elegidos.
  
    # Elimina los dos clusters elegidos de la matriz de distancias.
    
    # Actualiza la matriz de distancias ingresando el nuevo cluster.

    pass

  
  def update_matrix(self, cluster):

    if self.linkage == "centroid":
      pass

    elif self.linkage == "single":
      pass

  def get_distance(self, vector1, vector2):
    if self.distance == "Euclidean": 
      pass
    
    elif self.distance == "Cosine":
      pass

Utiliza la clase para realizar la aglomeración con los datos preprocesados de spam:

# 4. Comparación con distintos parámetros (1 punto)

En esta parte deberás comparar distintas configuraciones de tu algoritmo de *clustering* y concluir cual de estas es la mejor.

Una forma de comparar *clusters* es a partir de su *silhouette score*. Este mide cuán similar es un objeto a su propio *cluster* (cohesión) en comparación con otros *clusters* (separación). Completa el siguiente código utilizando la función `silhouette_score` de `sklearn.metrics`.

NOTA: debes adaptar la estructura de clusters retornada por `AgglomerativeClustering` de tal forma que pueda ser utilizada como los `labels` que recibe `silhouette_score` ([documentación](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)).

In [ ]:
from sklearn.metrics import silhouette_score

def messages_silhouette_score(X, clusters):
  pass

Ahora realiza una busqueda de hiperparámetros para encontrar la configuración que retorne el mejor *silhouette_score*. Como mínimo debes probar todas las combinaciones posibles de los siguientes parámetros:

*   ***Linkage***: centroid y single.
*   ***Distance***: euclidean y cosine.

Hecho esto, responde las siguientes preguntas. Debes fundamentar todas tus respuestas con los resultados obtenidos en la búsqueda de hiperparametros.
1.   ¿Cual configuración fue la mejor? 
2.   ¿Que métrica de distancia da mejores resultados? (puedes comparar las métricas fijando un valor) 
3.   ¿Que relación observas entre el método de enlace y la métrica de distancia utilizada? **Justifica**.


# 5. Visualización (1 punto)

Los [dendrogramas](https://es.wikipedia.org/wiki/Dendrograma) son una forma muy útil de visualizar el funcionamiento de los algoritmos de *clustering* aglomerativo. Para completar esta sección debes generar un dendrograma a partir de tu mejor configuración de `AgglomerativeClustering`. Para esto debes utilizar la función `dendrogram` del módulo `cluster.hierarchy` de scipy.

<br>
<center>
<img src="https://docs.scipy.org/doc/scipy/_images/scipy-cluster-hierarchy-dendrogram-1_00.png" width="400"/>

Fuente: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html

</center>

NOTA: Debes investigar que formato tiene la matriz `Z` (*linkage matrix*) que recibe `dendrogram` y adaptar el output de tu algoritmo acordemente. Está estrictamente prohibido obtener `Z` a partir de la función `linkage` del módulo `cluster.hierarchy`. Puedes modificar la clase `AgglomerativeClustering` si lo consideras necesario.

In [ ]:
from scipy.cluster.hierarchy import dendrogram